In [1]:
import nengo
from nengo.utils.matplotlib import rasterplot
from nengo_extras.plot_spikes import (
    cluster, merge, plot_spikes, preprocess_spikes, sample_by_variance)

import nengo_spa as spa
import numpy as np
spa.modules.basalganglia.BasalGanglia.sBCBG = True
import matplotlib
import matplotlib.style
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.rcParams['lines.linewidth'] = 2.
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = [10.,6.]

value = .4
grey = matplotlib.colors.to_hex((value, value, value))
plt.rcParams['axes.edgecolor']=grey
plt.rcParams['xtick.color'] = grey
plt.rcParams['ytick.color'] = grey
plt.rcParams['grid.color'] = grey


plt.rcParams['axes.titlepad']=10.
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['axes.titlesize'] = 17.
plt.rcParams['axes.labelsize'] = 18

plt.rcParams['errorbar.capsize'] = 5


import nengo_ocl
import sBCBG





# Number of dimensions for the Semantic Pointers
dimensions = 64
n_per_dim = 100

sequence = ['RED', 'BLUE', 'YELLOW', 'WHITE', 'BLACK', 'ORANGE'][:2]


class Input:

    def __init__(self, sequence, duration, dt=0.001):
        self.sequence = sequence
        self.index = 0
        self.duration = duration
        self.dt = dt
        self.count = 0
        
    def step(self, t):
        elt = self.sequence[self.index]
        if self.count > self.duration:
            self.count = 0
            self.index = (self.index + 1) % len(self.sequence)
        self.count += self.dt
        return elt
    


nbCh 1


## Functions to compute the power spectrum and the oscillation index

In [2]:
min_freq=15
max_freq=30

#------------------------------------------
# Computes Oscillation index from Kumar 2011
# Integral from a to b (spectrum) / Integral from 0 to (sampling freq. / 2) (spectrum)
# Note : the given spectrum must already be truncated (i.e. with x < sampling freq. / 2)
# returns 0 if denominator == 0
#------------------------------------------
def OscIndex(PS, freqs, a, b):
  tot = PS.sum()
  
  if tot != 0:
    idx = np.argsort(freqs)
    posi_spectrum = np.where((freqs[idx]>a) & (freqs[idx]<b)) # restrict the analysis to freqs [a-b] Hz
    return PS[posi_spectrum].sum()/tot
  else:
   return 0

def PS(data, N, sim, linestyle='-', OI=True, highlight=True, color='black'):
    
    time_step = sim.model.dt
    
    hanning = False
    if hanning:
        data = data * np.hanning(len(data))

    ps = np.abs(np.fft.fft(data))**2
    freqs = np.fft.fftfreq(data.size, time_step)
    
    idx = np.argsort(freqs)
    posi_spectrum = np.where((freqs[idx]>1) & (freqs[idx]<100)) # restrict the analysis to specified freqs  
    
    OI_value = OscIndex(ps[idx][posi_spectrum], freqs[idx][posi_spectrum], min_freq, max_freq)
    OI_legend = '' if not OI else ' (Osc. index: '+str(round(OI_value,2))+')'
    plt.plot(freqs[idx][posi_spectrum], ps[idx][posi_spectrum], color=color, linewidth=2., label=N+OI_legend, linestyle=linestyle)
    if highlight:
        plt.axvspan(min_freq, max_freq, color=(1,0,0,.2))
    #plt.yticks([0])
    plt.ylabel('Power')

    
    plt.xlabel('Frequency [Hz]')
    #plt.gca().get_yaxis().set_visible(False)
    
    return OI_value
        
    
    



## Simulation parameters (Parkinson vs normal)

The method for simulating Parkinson's disease (PD) is justified here:

[1] Beta-Band Oscillations without Pathways: the opposing Roles of D2 and D5 Receptors
Jean F. Liénard, Ignasi Cos, Benoît Girard
bioRxiv 161661; doi: https://doi.org/10.1101/161661

In [ ]:
normal = {
'splitGPe':                  False,
'nbCh':                          1, # number of concurrent channels to simulate
'LG14modelID':                   0, # LG 2014 parameterization used (default: model #9)

'nbMSN':                     2644., # population size (default: 1/1000 of the BG)
'nbFSI':                       53., # ^
'nbSTN':                        8., # ^
'nbGPe':                       25., # ^
'nbArky':                       5., # part of the GPe which projects to the striatum
'nbProt':                      20., # part of the GPe which projects to the STN and GPi/SNr
'nbGPi':                       14., # ^
'nbCSN':                     3000., # ^
'nbPTN':                      100., # ^
'nbCMPf':                       9., # ^

'GCSNMSN':           1., # defining connection types for channel-based models (focused or diffuse) based on LG14 - refer to this paper for justification
'GPTNMSN':           1., # ^
'GCMPfMSN':          1., # ^
'GMSNMSN':           1., # ^
'GFSIMSN':           1., # ^
'GSTNMSN':           1., # ^
'GGPeMSN':           1., # ^
'GArkyMSN':          1., # ^

'GCSNFSI':           1., # ^
'GPTNFSI':           1., # ^
'GCMPfFSI':          1., # ^
'GFSIFSI':           1., # ^
'GSTNFSI':           1., # ^
'GGPeFSI':           1., # ^
'GArkyFSI':          1., # ^

'GPTNSTN':           1., # ^
'GCMPfSTN':          1., # ^
'GGPeSTN':           1., # ^
'GProtSTN':          1., # ^

'GCMPfGPe':          1., # ^
'GMSNGPe':           1., # ^
'GSTNGPe':           1., # ^
'GGPeGPe':           1., # ^

'GCMPfArky':         1., # ^
'GMSNArky':          1., # ^
'GSTNArky':          1., # ^
'GArkyArky':         1/5., # ^
'GProtArky':         4/5., # ^

'GCMPfProt':         1., # ^
'GMSNProt':          1., # ^
'GSTNProt':          1., # ^
'GArkyProt':         1/5., # ^
'GProtProt':         4/5., # ^

'GCMPfGPi':          1., # ^
'GMSNGPi':           1., # ^
'GSTNGPi':           1., # ^
'GGPeGPi':           1., # LG14: no data available to decide; setting to diffuse improve selection properties
'GProtGPi':          1., #

'IeMSN':                        29.75, # tonic input currents (default: no input current)
'IeFSI':                        1., # ^
'IeSTN':                        9.25, # ^
'IeGPe':                        15., # ^
'IeArky':                       0., # ^
'IeProt':                       0., # ^
'IeGPi':                        15., # ^

"simulator": "Nengo",

# There are 3 different format for setting the inDegreeXY (=number of different incoming neurons from X that target one neuron in Y)
# - 'inDegreeAbs': specify the absolute number of different input neurons from X that target one neuron of Y --- be careful when using this setting, as the population size varies widly between the striatum and the downstream nuclei
# - 'outDegreeAbs': specify the absolute number of contacts between an axon from X to each dendritic tree in Y
# - 'outDegreeCons': specify the constrained number of contacts between an axon from X to each dendritic tree in Y as a fraction between 0 (minimal number of contacts to achieve required axonal bouton counts) and 1 (maximal number of contacts with respect to population numbers)

'RedundancyType':   'outDegreeAbs', # by default all axons are hypothesized to target each dendritic tree at 3 different locations
'redundancyCSNMSN':              3, # ^
'redundancyPTNMSN':              3, # ^
'redundancyCMPfMSN':             3, # ^
'redundancyMSNMSN':              3, # ^
'redundancyFSIMSN':              3, # ^
'redundancySTNMSN':              3, # ^
'redundancyGPeMSN':              3, # ^
'redundancyArkyMSN':             3, # ^

'redundancyCSNFSI':              3, # ^
'redundancyPTNFSI':              3, # ^
'redundancyCMPfFSI':             3, # ^
'redundancyFSIFSI':              3, # ^
'redundancySTNFSI':              3, # ^
'redundancyGPeFSI':              3, # ^
'redundancyArkyFSI':             3, # ^

'redundancyPTNSTN':              3, # ^
'redundancyCMPfSTN':             3, # ^
'redundancyGPeSTN':              3, # ^
'redundancyProtSTN':              3, # ^

'redundancyCMPfGPe':             3, # ^
'redundancySTNGPe':              3, # ^
'redundancyMSNGPe':              3, # ^
'redundancyGPeGPe':              3, # ^

'redundancyCMPfArky':            3, # ^
'redundancySTNArky':             3, # ^
'redundancyMSNArky':             3, # ^
'redundancyArkyArky':            3, # ^
'redundancyProtArky':            3, # ^

'redundancyCMPfProt':            3, # ^
'redundancySTNProt':             3, # ^
'redundancyMSNProt':             3, # ^
'redundancyArkyProt':            3, # ^
'redundancyProtProt':            3, # ^

'redundancyCMPfGPi':             3, # ^
'redundancyMSNGPi':              3, # ^
'redundancySTNGPi':              3, # ^
'redundancyGPeGPi':              3, # ^
'redundancyProtGPi':             3, # ^

'cTypeCSNMSN':           'focused', # defining connection types for channel-based models (focused or diffuse) based on LG14 - refer to this paper for justification
'cTypePTNMSN':           'focused', # ^
'cTypeCMPfMSN':          'diffuse', # ^
'cTypeMSNMSN':           'diffuse', # ^
'cTypeFSIMSN':           'diffuse', # ^
'cTypeSTNMSN':           'diffuse', # ^
'cTypeGPeMSN':           'diffuse', # ^
'cTypeArkyMSN':          'diffuse', # ^

'cTypeCSNFSI':           'focused', # ^
'cTypePTNFSI':           'focused', # ^
'cTypeCMPfFSI':          'diffuse', # ^
'cTypeFSIFSI':           'diffuse', # ^
'cTypeSTNFSI':           'diffuse', # ^
'cTypeGPeFSI':           'diffuse', # ^
'cTypeArkyFSI':          'diffuse', # ^

'cTypePTNSTN':           'focused', # ^
'cTypeCMPfSTN':          'diffuse', # ^
'cTypeGPeSTN':           'focused', # ^
'cTypeProtSTN':          'focused', # ^

'cTypeCMPfGPe':          'diffuse', # ^
'cTypeMSNGPe':           'focused', # ^
'cTypeSTNGPe':           'diffuse', # ^
'cTypeGPeGPe':           'diffuse', # ^

'cTypeCMPfArky':         'diffuse', # ^
'cTypeMSNArky':          'focused', # ^
'cTypeSTNArky':          'diffuse', # ^
'cTypeArkyArky':         'diffuse', # ^
'cTypeProtArky':         'diffuse', # ^

'cTypeCMPfProt':         'diffuse', # ^
'cTypeMSNProt':          'focused', # ^
'cTypeSTNProt':          'diffuse', # ^
'cTypeArkyProt':         'diffuse', # ^
'cTypeProtProt':         'diffuse', # ^

'cTypeCMPfGPi':          'diffuse', # ^
'cTypeMSNGPi':           'focused', # ^
'cTypeSTNGPi':           'diffuse', # ^
'cTypeGPeGPi':           'diffuse', # LG14: no data available to decide; setting to diffuse improve selection properties
'cTypeProtGPi':          'diffuse', #

'parrotCMPf' :                True, # Should the CMPf be simulated using parrot neurons?
'stochastic_delays':          None, # If specified, gives the relative sd of a clipped Gaussian distribution for the delays
# For convenience, a few simulator variables are also set here
'whichTest':          'testFullBG', # task to be run (default: test the plausibility through deactivation simulations)
'nestSeed':                     20, # nest seed (affects input poisson spike trains)
'pythonSeed':                   10, # python seed (affects connection map)
'nbcpu':                         1, # number of CPUs to be used by nest
'durationH':                  '08', # max duration of a simulation, used by Sango cluster
'nbnodes':                     '1', # number of nodes, used by K computer
'tSimu':                     5000., # time duration of one simulation
}



PD = normal.copy()
G = 3.

PD.update({
'GPTNSTN':           G, # ^
'GCMPfSTN':          G, # ^
'GGPeSTN':           G, # ^
'GProtSTN':          G, # ^

'GCMPfGPe':          G, # ^
'GMSNGPe':           G, # ^
'GSTNGPe':           G, # ^
'GGPeGPe':           G, # ^
})




In [ ]:
n_trials = 10
OI_results = np.zeros((2,2,n_trials)) # STN/GPe, normal/PD, trials

## Run simulations

In [ ]:
for trial in range(n_trials):
    with nengo.Network() as model_normal:

        sBCBG.nengo_instantiate(1, model_normal, normal)

        model_normal.CSN_in = nengo.Probe(model_normal.pops['CSN'].neurons, 'input')
        model_normal.CSN = nengo.Probe(model_normal.pops['CSN'].neurons)
        model_normal.MSN = nengo.Probe(model_normal.pops['MSN'].neurons)
        model_normal.STN = nengo.Probe(model_normal.pops['STN'].neurons)
        model_normal.GPe = nengo.Probe(model_normal.pops['GPe'].neurons)
        model_normal.GPi = nengo.Probe(model_normal.pops['GPi'].neurons)

    with nengo.Simulator(model_normal) as sim_normal:
        sim_normal.run(2)

    OI_results[0, 0, trial] = PS(sim_normal.data[model_normal.STN].sum(axis=1)[-500:], '', sim_normal)
    OI_results[1, 0, trial] = PS(sim_normal.data[model_normal.GPe].sum(axis=1)[-500:], '', sim_normal)


In [ ]:
for trial in range(n_trials):
    with nengo.Network() as model_PD:

        sBCBG.nengo_instantiate(1, model_PD, PD)


        model_PD.CSN_in = nengo.Probe(model_PD.pops['CSN'].neurons, 'input')
        model_PD.CSN = nengo.Probe(model_PD.pops['CSN'].neurons)
        model_PD.MSN = nengo.Probe(model_PD.pops['MSN'].neurons)
        model_PD.STN = nengo.Probe(model_PD.pops['STN'].neurons)
        model_PD.GPe = nengo.Probe(model_PD.pops['GPe'].neurons)
        model_PD.GPi = nengo.Probe(model_PD.pops['GPi'].neurons)

    with nengo.Simulator(model_PD) as sim_PD:
        sim_PD.run(2)

    OI_results[0, 1, trial] = PS(sim_PD.data[model_PD.STN].sum(axis=1)[-500:], '', sim_PD)
    OI_results[1, 1, trial] = PS(sim_PD.data[model_PD.GPe].sum(axis=1)[-500:], '', sim_PD)


## Plot power spectrums and oscillation indices (see poster)

In [ ]:
sims = [sim_normal, sim_PD]
nets = [model_normal, model_PD]
desc = ['Control', 'Parkinson']
linestyles = ['-','-']
colors = ['black', (1,0,0,.7)]

scaling = .75
plt.figure(figsize=(18*scaling,6*scaling))


gs = gridspec.GridSpec(1, 5, width_ratios=[7, 2, 1.5, 7, 2]) 

for i in range(len(sims)):

    plt.subplot(gs[0])
    print('\n\nSTN '+desc[i])
    PS(sims[i].data[nets[i].STN].sum(axis=1)[-400:], desc[i], sims[i], linestyle=linestyles[i], OI=False, highlight=i==True, color=colors[i])
    plt.xticks([0,15,30,50,75,100])
    plt.title('STN', fontdict={'fontsize':25})
    plt.legend(fontsize=16)
    
    

    plt.subplot(gs[3])
    print('\n\nGPe '+desc[i])
    PS(sims[i].data[nets[i].GPe].sum(axis=1)[-400:], desc[i], sims[i], linestyle=linestyles[i], OI=False, highlight=i==True, color=colors[i])
    plt.xticks([0,15,30,50,75,100])
    plt.title('GPe', fontdict={'fontsize':25})
    plt.legend(fontsize=16)

    '''plt.subplot(133)
    print('\n\nGPi '+desc[i])
    PS(sims[i].data[nets[i].GPi].sum(axis=1)[-500:], desc[i], sims[i], linestyle=linestyles[i])
    plt.title('GPi')
    plt.legend()'''



plt.subplot(gs[1])
#plt.title('15-30Hz', fontsize=15)
bars = plt.bar(range(len(desc)), OI_results[0,:,:].mean(axis=1), yerr=[(0,0),OI_results[0,:,:].std(axis=1)], edgecolor='black', color=colors, linewidth=[2, 2], linestyle='-', align='edge', width=.8)
#bars[0].set_linestyle('--')
plt.gca().set_facecolor((1,0,0,.2))

plt.xticks(range(len(desc)), desc, fontsize=30)
plt.xticks([])
plt.tick_params(axis='y', which='both', labelleft='off', labelright='on', length=0)
#plt.yticks(color='black', fontsize=16)
plt.ylim(0, .58)
h = plt.ylabel('Relative power of 15-30Hz', labelpad=25)
h.set_rotation(-90)
plt.gca().yaxis.set_label_position("right")


plt.subplot(gs[4])
#plt.title('15-30Hz', fontsize=15)
bars = plt.bar(range(len(desc)), OI_results[1,:,:].mean(axis=1), yerr=[(0,0),OI_results[1,:,:].std(axis=1)], edgecolor='black', color=colors, linewidth=[2, 2], linestyle='-', align='edge', width=.8)
#bars[0].set_linestyle('--')
plt.gca().set_facecolor((1,0,0,.2))
plt.xticks(range(len(desc)), desc, fontsize=30)
plt.xticks([])
plt.tick_params(axis='y', which='both', labelleft='off', labelright='on', length=0)
#plt.yticks(color='black', fontsize=16)
plt.ylim(0, .58)
h = plt.ylabel('Relative power of 15-30Hz', labelpad=25)
h.set_rotation(-90)
plt.gca().yaxis.set_label_position("right")



plt.savefig('PS', format='svg')
plt.show()
